In [1]:
from enum import Enum

class Mode(Enum):
    OFF = 1
    ON = 2
    FOLLOW = 3
    MOVE = 4

class Track(Enum):
    NO = 1
    YES = 2

class State(Enum):
    PERFECT = 1
    BATTERYERROR = 2
    TEMPPB = 3
    TEMPERROR = 4
    ERROR = 5
    
class Weather(Enum):
    CLEAR = 1
    RAIN = 2
    WIND = 3
    
    

In [2]:
import random
from datetime import datetime

class Group:
    def __init__(self, id, x, y):
        self.id = id
        self.x = x
        self.y = y
        self.number_people = random.randint(100,1000)
        self.mode = Mode.ON
        self.number_angry = 0
        self.trackable = Track.NO
        self.tracked = Track.NO
        self.date = str(datetime.now())[:19]
        self.timestamp = datetime.timestamp(datetime.now())
        
    def update(self): 
        #80% of chance that the group does something or not 
        if random.randint(1,10) > 2 :
            
            #80% of chance that the group moves 
            if random.randint(1,10) > 2 :
                self.mode = Mode.MOVE
                self.x += random.uniform(-3, 3)
                self.y += random.uniform(-3, 3)
            else :
                self.mode = Mode.ON
                
            #80% of chance that number of angry people will change  
            if random.randint(1,10) > 2 :
                
                self.number_angry = random.randint(0, self.number_people)
                if self.number_angry >= int(self.number_people/4):
                    self.trackable = Track.YES
                else :
                    self.tracked = Track.NO
                    self.trackable = Track.NO
                    
            self.timestamp = datetime.timestamp(datetime.now())
            return self.profil()
        
        #Does nothing 
        else :
            self.mode = Mode.ON
            self.timestamp = datetime.timestamp(datetime.now())
            return self.profil()
                 
            
    def profil(self):
        info = dict()
        info["timestamp"] = self.timestamp
        info["group_id"] = self.id
        info["posX"] = self.x
        info["posY"] = self.y
        info["mode"] = self.mode.name
        info["trackable"] = self.trackable.name
        info["tracked"] = self.tracked.name
        info["people"] = self.number_people
        info["people_angry"] = self.number_angry
        info["date_creation"] = self.date
        
        return info

In [3]:
from datetime import datetime

class Drone:
    def __init__(self, id, x, y):
        self.date = str(datetime.now())[:19]
        self.timestamp = datetime.timestamp(datetime.now())
        self.x = x
        self.y = y
        self.state = State.PERFECT
        self.temp = 20
        self.weather = Weather.CLEAR
        self.mode = Mode.ON
        self.id = id
        self.speed = 2
        self.battery = 100
        
        self.g = None
        self.groupId = None
        self.peolpe = None
        self.angry = None
        self.gx = None
        self.gy = None
        self.trackable = Track.NO
        self.tracked = None
    
    def detect(self, group):
        if group.trackable.name == "YES":
            group.tracked = Track.YES
            
            self.g = group
            self.groupId = group.id
            self.peolpe = group.number_people
            self.angry = group.number_angry
            self.gx = group.x
            self.gy = group.y
            self.trackable = group.trackable
            self.tracked = group.tracked
            self.mode.name == "FOLLOW"
            
            return self.update()
        else :
            self.mode.name == "ON"
            return self.update()
        
        
    def update(self):
        if self.battery <= 0 :
            self.mode = Mode.OFF
            self.state = State.BATTERYERROR
            self.timestamp = datetime.timestamp(datetime.now()) 
            return self.profil()

        if self.x == 0 and self.y == 0:
            self.temp = 0
        if self.x > 0 and self.y > 0:
            self.temp = 20
        if self.x < 0 and self.y > 0:
            self.temp = 10
        if self.x > 0 and self.y < 0:
            self.temp = 0
        if self.x < 0 and self.y < 0:
            self.temp = -10
        
        if self.weather.name == "CLEAR":
            if random.randint(0, 9) == 0:
                if random.randint(0, 1) == 0:
                    self.weather = Weather.RAIN
                else:
                    self.weather = Weather.WIND   
                    
        if self.weather.name == "WIND":
            if random.randint(0, 9) == 0:
                if random.randint(0, 1) == 0:
                    self.weather = Weather.RAIN
                else:
                    self.weather = Weather.CLEAR  
                    
        if self.weather.name == "RAIN":
            if random.randint(0, 9) == 0:
                if random.randint(0, 1) == 0:
                    self.weather = Weather.CLEAR
                else:
                    self.weather = Weather.WIND
            
        if self.weather.name == "RAIN":
            self.temp -= random.randint(0, 10)

        if self.weather.name == "CLEAR":
            self.temp += random.randint(0, 10)
        
        if self.weather.name == "WIND":
            self.battery -= 1
            
        if random.randint(0, 300 - abs(self.temp)) == 0:
            self.state = State.TEMPPB

        if random.randint(0, 5) == 0 and self.state.name == "TEMPPB":
            self.state = State.TEMPERROR
            self.mode = Mode.OFF
            self.timestamp = datetime.timestamp(datetime.now()) 
            return self.profil()
        
        if random.randint(0, 1) == 0:
            self.state = State.PERFECT
            
        #if battery > 50%
        if self.battery <= 50 :
            #print ("**********************1**********************") 
            self.mode = Mode.ON
            distanceX = self.x
            distanceY = self.y
            distanceX2 = self.x
            distanceY2 = self.y
            
            #print ("**********************1.0**********************X", distanceX) 
            #print ("**********************1.0**********************Y", distanceY) 

            if (abs(distanceX) + abs(distanceY) > 5) :
                a = abs(distanceX) + abs(distanceY)
                if a < 1 :
                    a = 1

                distanceX = (-distanceX / a) * self.speed  
                distanceY = (-distanceY / a) * self.speed
                
                #print ("**********************1.1**********************X", distanceX) 
                #print ("**********************1.1**********************Y", distanceY) 
                
                if self.battery > 1 :
                    #print ("**********************1.2**********************MOVE") 
                    self.x += distanceX
                    self.y += distanceY
                    self.battery -= self.speed
                #if it has only 1% of battery
                else : 
                    #print ("**********************1.3**********************DEAD") 
                    self.battery -= self.speed/2
                    self.mode = Mode.OFF
                    self.state = State.BATTERYERROR
                    self.timestamp = datetime.timestamp(datetime.now()) 
                    return self.profil()
                    
            if (abs(distanceX2) + abs(distanceY2) <= 5) :
                #print ("**********************1.4**********************RECHARGE") 
                self.battery = 100
        
        #if trackable
        if self.trackable.name == "YES" and self.battery > 50 :  
            self.mode = Mode.FOLLOW
            distanceX = self.x - self.gx
            distanceY = self.y - self.gy

            if (distanceX + distanceY != 0) :
                a = abs(distanceX) + abs(distanceY)
                if a < 1 :
                    a = 1

                distanceX = (-distanceX / a) * self.speed  
                distanceY = (-distanceY / a) * self.speed 

                self.x += distanceX
                self.y += distanceY
                self.battery -= self.speed
        else : 
            #Back to home
            self.mode = Mode.ON
            #print ("**********************2**********************")
            distanceX = self.x
            distanceY = self.y
            distanceX2 = self.x
            distanceY2 = self.y
            #print ("**********************2.0**********************X", distanceX) 
            #print ("**********************2.0**********************Y", distanceY) 

            if (abs(distanceX) + abs(distanceY) > 5) :
                a = abs(distanceX) + abs(distanceY)
                if a < 1 :
                    a = 1

                distanceX = (-distanceX / a) * self.speed  
                distanceY = (-distanceY / a) * self.speed
                
                #print ("**********************2.1**********************X", distanceX) 
                #print ("**********************2.1**********************Y", distanceY) 

                if self.battery > 1 :
                    #print ("**********************2.2**********************MOVE") 
                    self.x += distanceX
                    self.y += distanceY
                    self.battery -= self.speed
                else : 
                    #print ("**********************2.3**********************DEAD") 
                    self.battery -= self.speed/2
                    self.mode = Mode.OFF
                    self.state = State.BATTERYERROR
                    self.timestamp = datetime.timestamp(datetime.now()) 
                    return self.profil()
                
            if (abs(distanceX2) + abs(distanceY2) <= 5) : 
                #print ("**********************2.4**********************RECHARGE") 
                self.battery = 100
                    
        self.timestamp = datetime.timestamp(datetime.now())        
        return self.profil()

    def profil(self):
        info = dict()
        info["timestamp"] = self.timestamp
        info["drone_id"] = self.id
        info["temp"] = self.temp
        info["weather"] = self.weather.name
        info["state"] = self.state.name
        info["posX"] = self.x
        info["posY"] = self.y
        info["mode"] = self.mode.name
        info["battery"] = self.battery
        info["groupId"] = self.groupId
        info["groupX"] = self.gx
        info["groupY"] = self.gy
        info["people"] = self.peolpe
        info["people_angry"] = self.angry
        info["date_creation"] = self.date
        
        
        return info

In [4]:
drones = []
groups = []
D_id = 1
G_id = 1
nbDrones = 200
nbGroups = 400
fileJson = []

#Creating drones
for i in range (0, nbDrones):
    drones.append(Drone(D_id, 0, 0))
    json = drones[i].profil()
    fileJson.append(json)
    D_id += 1
    
#Creating groups
for i in range (0, nbGroups):
    groups.append(Group(G_id, random.uniform(-5, 5), random.uniform(-5, 5)))
    groups[i].profil()
    #json = groups[i].profil()
    #fileJson.append(json)
    G_id += 1

    
#Updating all groups
for Group in groups :
    Group.update()
    #json = Group.update()
    #fileJson.append(json)

In [5]:
for i in range (0, 500) :
    for Drone in drones :
        #if drone already has a group, check if the group is still trackable
        if drones:
            if Drone.mode.name == "FOLLOW":
                json = Drone.detect(Drone.g) #Automatic update and check if the group is still trackable
                fileJson.append(json)
                if Drone.mode.name == "OFF" :
                    drones.remove(Drone)

            else : #drone is ON mode
                for Group in groups :
                    if Group.tracked.name == "NO" :
                        json = Drone.detect(Group)
                        fileJson.append(json)
                        if Drone.mode.name == "OFF" :
                            drones.remove(Drone)
                            break
                    if Drone.mode.name == "FOLLOW":
                        break

    for Group in groups :
        Group.update()

In [6]:
import json

def writeToJSONFile(path, fileName, data):
    filePathNameWExt = './' + path + '/' + fileName + '.json'
    with open(filePathNameWExt, 'w') as fp:
        json.dump(data, fp)
       
    
path = './'
writeToJSONFile(path, 'file', fileJson)   

In [7]:
fileJson

[{'timestamp': 1554429465.149462,
  'drone_id': 1,
  'temp': 20,
  'weather': 'CLEAR',
  'state': 'PERFECT',
  'posX': 0,
  'posY': 0,
  'mode': 'ON',
  'battery': 100,
  'groupId': None,
  'groupX': None,
  'groupY': None,
  'people': None,
  'people_angry': None,
  'date_creation': '2019-04-05 03:57:45'},
 {'timestamp': 1554429465.14949,
  'drone_id': 2,
  'temp': 20,
  'weather': 'CLEAR',
  'state': 'PERFECT',
  'posX': 0,
  'posY': 0,
  'mode': 'ON',
  'battery': 100,
  'groupId': None,
  'groupX': None,
  'groupY': None,
  'people': None,
  'people_angry': None,
  'date_creation': '2019-04-05 03:57:45'},
 {'timestamp': 1554429465.149499,
  'drone_id': 3,
  'temp': 20,
  'weather': 'CLEAR',
  'state': 'PERFECT',
  'posX': 0,
  'posY': 0,
  'mode': 'ON',
  'battery': 100,
  'groupId': None,
  'groupX': None,
  'groupY': None,
  'people': None,
  'people_angry': None,
  'date_creation': '2019-04-05 03:57:45'},
 {'timestamp': 1554429465.149508,
  'drone_id': 4,
  'temp': 20,
  'weathe

temperature en 4
l'hémisphère en 2
batterie faible defaillance 

In [8]:
import pandas as pd
content = pd.read_json("./file.json", lines=True)
content

,0,1,2,3,4,5,6,7,8,9,...,87452,87453,87454,87455,87456,87457,87458,87459,87460,87461
0,"{'timestamp': 1554429465.149462, 'drone_id': 1...","{'timestamp': 1554429465.14949, 'drone_id': 2,...","{'timestamp': 1554429465.149499, 'drone_id': 3...","{'timestamp': 1554429465.149508, 'drone_id': 4...","{'timestamp': 1554429465.149516, 'drone_id': 5...","{'timestamp': 1554429465.149524, 'drone_id': 6...","{'timestamp': 1554429465.149537, 'drone_id': 7...","{'timestamp': 1554429465.14955, 'drone_id': 8,...","{'timestamp': 1554429465.149561, 'drone_id': 9...","{'timestamp': 1554429465.149574, 'drone_id': 1...",...,"{'timestamp': 1554429468.023026, 'drone_id': 1...","{'timestamp': 1554429468.023038, 'drone_id': 1...","{'timestamp': 1554429468.023052, 'drone_id': 1...","{'timestamp': 1554429468.023066, 'drone_id': 1...","{'timestamp': 1554429468.023078, 'drone_id': 1...","{'timestamp': 1554429468.02309, 'drone_id': 18...","{'timestamp': 1554429468.023103, 'drone_id': 1...","{'timestamp': 1554429468.023116, 'drone_id': 1...","{'timestamp': 1554429468.023127, 'drone_id': 1...","{'timestamp': 1554429468.02314, 'drone_id': 19..."


In [9]:
pd.read_json(json.dumps(fileJson)).to_csv('fileCSV.csv')